# EV Sensitivity (Location)

In this notebook I keep EV size fixed (for example 7 kW) and move the EV to different locations.

Locations:
- source bus
- mid feeder bus
- far end bus
- weakest bus (lowest baseline voltage)

For each location I save:
- topology plot with EV marked
- voltage profile comparison (baseline vs EV)
- simple metrics (min voltage and violation count)


In [ ]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

using PowerModelsDistribution
using DataFrames
using Ipopt
using JuMP
using LinearAlgebra
using Plots
using Graphs
using CairoMakie
using Statistics

const PMD = PowerModelsDistribution

file = "../data/Three-wire-Kron-reduced/network_1/Feeder_1/Master.dss"
fig_dir = "../results/figures"
mkpath(fig_dir)

  Activating project at `/mnt/c/Users/auc009/OneDrive - CSIRO/Documents/power-models-distribution/pmd_ev_experiments`


"../results/figures"

## Reuse helper functions

Copy the helper function block from 02_single_ev.ipynb into this notebook.

I keep the notebooks self contained for now.

In [2]:
function build_lines_df(eng)
    line_dict = get(eng, "line", Dict())
    rows = NamedTuple[]
    for (id, ln) in line_dict
        bus1 = get(ln, "bus1", get(ln, "f_bus", nothing))
        bus2 = get(ln, "bus2", get(ln, "t_bus", nothing))
        if bus1 === nothing || bus2 === nothing
            continue
        end
        b1 = split(String(bus1), ".")[1]
        b2 = split(String(bus2), ".")[1]
        L = float(get(ln, "length", 1.0))
        push!(rows, (Bus1=b1, Bus2=b2, Length=L))
    end
    return DataFrame(rows)
end

function build_graph_and_weights(lines_df::DataFrame)
    buses = unique(vcat(lines_df.Bus1, lines_df.Bus2))
    sort!(buses)
    b2i = Dict(b => i for (i,b) in enumerate(buses))
    i2b = Dict(i => b for (b,i) in b2i)

    g = Graph(length(buses))
    w = Dict{Tuple{Int,Int}, Float64}()

    for r in eachrow(lines_df)
        i = b2i[r.Bus1]
        j = b2i[r.Bus2]
        add_edge!(g, i, j)
        w[(i,j)] = r.Length
        w[(j,i)] = r.Length
    end
    return g, w, b2i, i2b
end

function dijkstra_distances(g::Graph, w::Dict{Tuple{Int,Int},Float64}, s::Int)
    n = nv(g)
    dist = fill(Inf, n)
    prev = fill(0, n)
    visited = falses(n)

    dist[s] = 0.0
    pq = [(0.0, s)]

    while !isempty(pq)
        sort!(pq, by=x->x[1])
        (d,u) = popfirst!(pq)
        if visited[u]
            continue
        end
        visited[u] = true

        for v in neighbors(g, u)
            nd = d + get(w, (u,v), 1.0)
            if nd < dist[v]
                dist[v] = nd
                prev[v] = u
                push!(pq, (nd, v))
            end
        end
    end
    return dist, prev
end

function tree_children(prev::Vector{Int})
    n = length(prev)
    children = [Int[] for _ in 1:n]
    for v in 1:n
        p = prev[v]
        if p != 0
            push!(children[p], v)
        end
    end
    return children
end

function tree_layout(prev::Vector{Int}, dist_km::Vector{Float64}, root::Int)
    children = tree_children(prev)
    y = fill(0.0, length(prev))
    next_y = Ref(0.0)

    function dfs(u::Int)
        if isempty(children[u])
            y[u] = next_y[]
            next_y[] += 1.0
        else
            for c in sort(children[u])
                dfs(c)
            end
            y[u] = mean(y[children[u]])
        end
    end

    dfs(root)
    x = copy(dist_km)
    return x, y
end

function plot_topology_with_markers(
    g::Graph, xpos::Vector{Float64}, ypos::Vector{Float64}, dist_km::Vector{Float64};
    source_idx::Int,
    ev_indices::Vector{Int}=Int[],
    title_str::String="Topology (colored by distance)",
    save_path::Union{Nothing,String}=nothing
)
    fig = Figure(size=(1000, 700))
    ax = Axis(fig[1,1], title=title_str)

    for e in edges(g)
        u = src(e); v = dst(e)
        lines!(ax, [xpos[u], xpos[v]], [ypos[u], ypos[v]], color=:gray50, linewidth=2)
    end

    scatter!(ax, xpos, ypos, color=dist_km, markersize=10)
    scatter!(ax, [xpos[source_idx]], [ypos[source_idx]], markersize=22, color=:black)
    text!(ax, "source", position=(xpos[source_idx], ypos[source_idx] + 0.6), align=(:center, :bottom))

    for (k, ev) in enumerate(ev_indices)
        scatter!(ax, [xpos[ev]], [ypos[ev]], markersize=22, color=:red)
        text!(ax, "EV$(k)", position=(xpos[ev], ypos[ev] + 0.6), align=(:center, :bottom))
    end

    hidedecorations!(ax)
    hidespines!(ax)

    if save_path !== nothing
        save(save_path, fig)
    end
    return fig
end

function busid_to_name_map(math)
    m = Dict{String,String}()
    for (id, b) in math["bus"]
        m[string(id)] = split(get(b, "name", string(id)), ".")[1]
    end
    return m
end

function extract_vm_by_busid(result)
    vm = Dict{String, Vector{Float64}}()
    for (id, b) in result["solution"]["bus"]
        vr = b["vr"]; vi = b["vi"]
        vm[string(id)] = sqrt.(vr.^2 .+ vi.^2)
    end
    return vm
end

function build_buses_dict_for_voltage_plot(id2name::Dict{String,String}, dist_km_byname::Dict{String,Float64}, vm_byid::Dict{String,Vector{Float64}})
    buses_dict = Dict{String, Dict{String,Any}}()
    for (id, vm) in vm_byid
        name = id2name[id]
        vma = length(vm) >= 1 ? vm[1] : NaN
        vmb = length(vm) >= 2 ? vm[2] : NaN
        vmc = length(vm) >= 3 ? vm[3] : NaN
        buses_dict[name] = Dict(
            "distance" => get(dist_km_byname, name, NaN),
            "vma" => [float(vma)],
            "vmb" => [float(vmb)],
            "vmc" => [float(vmc)]
        )
    end
    return buses_dict
end

function plot_voltage_along_feeder_snap(buses_dict, lines_df; t=1, vmin=0.9, vmax=1.1)
    p = plot(legend=false)
    xlabel!("Electrical distance from source (km)")
    ylabel!("Voltage magnitude (p.u.)")
    title!("Voltage drop along feeder (snapshot)")

    colors = [:blue, :red, :black]

    for r in eachrow(lines_df)
        b1 = r.Bus1
        b2 = r.Bus2
        for phase in 1:3
            d1 = buses_dict[b1]["distance"]
            d2 = buses_dict[b2]["distance"]
            v1 = phase==1 ? buses_dict[b1]["vma"][t] : phase==2 ? buses_dict[b1]["vmb"][t] : buses_dict[b1]["vmc"][t]
            v2 = phase==1 ? buses_dict[b2]["vma"][t] : phase==2 ? buses_dict[b2]["vmb"][t] : buses_dict[b2]["vmc"][t]
            if isfinite(d1) && isfinite(d2) && isfinite(v1) && isfinite(v2)
                plot!([d1, d2], [v1, v2], color=colors[phase], marker=:circle, markersize=1)
            end
        end
    end

    maxdist = maximum([b["distance"] for (k,b) in buses_dict if isfinite(b["distance"])])
    plot!([0, maxdist], [vmin, vmin], color=:red, linestyle=:dash)
    plot!([0, maxdist], [vmax, vmax], color=:red, linestyle=:dash)
    return p
end

function add_ev_load!(math, bus_id::String; phase::Int=1, P_kw::Float64=7.0, Q_kvar::Float64=0.0)
    sbase = get(math["settings"], "sbase", 1.0)
    P_pu = P_kw / (sbase * 1000)
    Q_pu = Q_kvar / (sbase * 1000)

    existing = parse.(Int, collect(keys(math["load"])))
    new_id = string(isempty(existing) ? 1 : maximum(existing) + 1)

    math["load"][new_id] = Dict(
        "bus" => bus_id,
        "connections" => [phase],
        "pd" => [P_pu],
        "qd" => [Q_pu],
        "status" => 1,
        "model" => "constant_power"
    )
    return new_id
end

function find_math_bus_id_by_name(id2name::Dict{String,String}, target_name::String)
    for (id, nm) in id2name
        if nm == target_name
            return id
        end
    end
    error("No math bus id found for name '$target_name'")
end

find_math_bus_id_by_name (generic function with 1 method)

In [3]:
eng4w = PMD.parse_file(file, transformations=[PMD.transform_loops!])
eng4w["settings"]["sbase_default"] = 1.0 * 1e3 / eng4w["settings"]["power_scale_factor"]

PMD.add_bus_absolute_vbounds!(eng4w, phase_lb_pu=0.9, phase_ub_pu=1.1, neutral_ub_pu=0.1)

lines_df = build_lines_df(eng4w)
g, w, b2i, i2b = build_graph_and_weights(lines_df)

source_name = "sourcebus"
source_idx = b2i[source_name]
dist_raw, prev = dijkstra_distances(g, w, source_idx)

scale_to_km = 1.0 / 1000
dist_km = dist_raw .* scale_to_km
xpos, ypos = tree_layout(prev, dist_km, source_idx)
dist_km_byname = Dict(i2b[i] => dist_km[i] for i in 1:length(dist_km) if isfinite(dist_km[i]))

math_base = PMD.transform_data_model(eng4w; multinetwork=false, kron_reduce=false, phase_project=false)
id2name = busid_to_name_map(math_base)

res_base = PMD.solve_mc_opf(math_base, IVRENPowerModel, Ipopt.Optimizer)
vm_base = extract_vm_by_busid(res_base)
buses_dict_base = build_buses_dict_for_voltage_plot(id2name, dist_km_byname, vm_base)

p_base = plot_voltage_along_feeder_snap(buses_dict_base, lines_df; vmin=0.9, vmax=1.1)
savefig(p_base, joinpath(fig_dir, "03_voltage_profile_baseline.png"))


[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 18 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'linecode.txt' on line 23 in 'Master.dss'
[ PowerModelsDistribution | Info ] : Redirecting to 'lines.txt' on line 24 in 'LineCode.txt'
[ PowerModelsDistribution | Info ] : Redirecting to 'loads.txt' on line 25 in 'Lines.txt'
[ PowerModelsDistribution | Info ] : Command 'solve' on line 30 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'closedi' on line 31 in 'Master.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : basemva=100 is the default value, you may want to adjust sbase_default for better convergence
┌ PowerModelsDistribution | Warning ] : basefreq=60.0 on line.line48 does not match circuit basefreq=50.0
└ @ PowerModelsDistribution ~/.julia/packages/PowerModelsDistribution/EOTxp/src/data_model/transformations/dss2eng.jl:422
┌ PowerModelsDistribution | Warning ] : ba


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.19, running with linear solver MUMPS 5.8.1.

Number of nonzeros in equality constraint Jacobian...:     8688
Number of nonzeros in inequality constraint Jacobian.:     1944
Number of nonzeros in Lagrangian Hessian.............:     2882

The Jacobian for the equality constraints contains an invalid number

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 0
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 1
Number of equality co

LoadError: MethodError: no method matching build_buses_dict_for_voltage_plot(::Dict{String, String}, ::Dict{SubString{String}, Float64}, ::Dict{String, Vector{Float64}})

[0mClosest candidates are:
[0m  build_buses_dict_for_voltage_plot(::Dict{String, String}, [91m::Dict{String, Float64}[39m, ::Dict{String, Vector{Float64}})
[0m[90m   @[39m [35mMain[39m [90m[4mIn[2]:150[24m[39m


In [ ]:
# Weakest bus by baseline min phase voltage
minV_base_byid = Dict(id => minimum(vm[1:min(3, length(vm))]) for (id, vm) in vm_base)
weakest_bus_id = first(keys(minV_base_byid))
weakest_val = minV_base_byid[weakest_bus_id]
for (id, v) in minV_base_byid
    if v < weakest_val
        weakest_val = v
        weakest_bus_id = id
    end
end
weakest_name = id2name[weakest_bus_id]

finite_mask = isfinite.(dist_km)
far_idx = argmax(dist_km .* finite_mask)
far_name = i2b[far_idx]

mid_target = 0.5 * maximum(dist_km[finite_mask])
mid_idx = argmin(abs.(dist_km .- mid_target) .+ (.!finite_mask) .* 1e9)
mid_name = i2b[mid_idx]

source_bus_id = string(math_base["bus_lookup"]["sourcebus"])
source_bus_name = id2name[source_bus_id]

println("source:  ", source_bus_name)
println("mid:     ", mid_name)
println("far:     ", far_name)
println("weakest: ", weakest_name, " (minV=", weakest_val, ")")


## Run EV sensitivity scenarios

I use a fixed EV size (7 kW) on phase A and move the EV location.


In [ ]:
P_ev_kw = 7.0
Q_ev_kvar = 0.0
ev_phase = 1

scenario_bus_names = Dict(
    "source"  => source_bus_name,
    "mid"     => mid_name,
    "far"     => far_name,
    "weakest" => weakest_name
)

rows = NamedTuple[]

for (label, bus_name) in scenario_bus_names
    println("\nScenario: ", label, " at bus ", bus_name)

    bus_id = find_math_bus_id_by_name(id2name, bus_name)

    math = deepcopy(math_base)
    add_ev_load!(math, bus_id; phase=ev_phase, P_kw=P_ev_kw, Q_kvar=Q_ev_kvar)

    res = PMD.solve_mc_opf(math, IVRENPowerModel, Ipopt.Optimizer)
    vm = extract_vm_by_busid(res)
    buses_dict = build_buses_dict_for_voltage_plot(id2name, dist_km_byname, vm)

    # Topology plot with EV marker
    ev_idx = b2i[bus_name]
    plot_topology_with_markers(
        g, xpos, ypos, dist_km;
        source_idx=source_idx,
        ev_indices=[ev_idx],
        title_str="Topology with EV at $(label)",
        save_path=joinpath(fig_dir, "03_topology_ev_$(label).png")
    )

    # Voltage profile comparison
    p = plot_voltage_along_feeder_snap(buses_dict, lines_df; vmin=0.9, vmax=1.1)
    plot!(p, p_base)
    savefig(p, joinpath(fig_dir, "03_voltage_profile_baseline_vs_ev_$(label).png"))

    # Metrics
    push!(rows, (
        scenario=label,
        bus=bus_name,
        status=string(res["termination_status"]),
        minV=min_phase_voltage(vm),
        violations=count_voltage_violations(vm; vmin=0.9, vmax=1.1)
    ))
end

sens_df = DataFrame(rows)
sens_df

## Notes

This notebook shows how sensitive the feeder voltage is to EV location.

The next notebook increases EV severity at a fixed location to find when voltage limits start to bind.
